In [67]:
import pandas as pd
import glob,os
import re


In [68]:
OMOP_FILES_PATH = 'codelists/OMOP/'

omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')
omop_relationships  = pd.read_csv(OMOP_FILES_PATH+'RELATIONSHIP.csv', sep='\t')
omop_concept_relationships  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT_RELATIONSHIP.csv', sep='\t')


omop_concepts

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1064190685.py:3: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,756315,metadata.metadata_type_concept_id,Metadata,CDM,Field,S,CDM1000,20210925,20991231,NaN
4,756316,metadata.name,Metadata,CDM,Field,S,CDM1001,20210925,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
1473553,33091,IQVIA CO-CONNECT,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473554,33092,IQVIA CO-CONNECT MIABIS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473555,33093,IQVIA CO-CONNECT TWINS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473556,33095,NHS Ethnic Category,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN


In [69]:
s='lung'
files = sorted( list( glob.glob("codelists/output/query_"+s+"*_snomed.csv") ) )
files


['codelists/output/query_lung_0_snomed.csv',
 'codelists/output/query_lung_1_snomed.csv',
 'codelists/output/query_lung_2_snomed.csv',
 'codelists/output/query_lung_3_snomed.csv']

In [70]:
for f in files:
    name = re.split('/|\.', f)[-2]
    input_snomed_list = pd.read_csv(f)

    input_snomed_list['conceptId'] = input_snomed_list['conceptId'].astype(str)
    omop_concepts['concept_code'] = omop_concepts['concept_code'].astype(str)
    omop_concepts['concept_id'] = omop_concepts['concept_id'].astype(str)

    omop_concepts_filtered = omop_concepts[ omop_concepts['concept_code'].isin(input_snomed_list['conceptId']) ]

    omop_concepts_filtered = omop_concepts_filtered[['concept_id','concept_name','vocabulary_id','concept_code']]
    omop_concepts_filtered

    #omop_relationships_filtered = omop_concept_relationships[ omop_concept_relationships['relationship_id'].isin(['Maps to','Subsumes'])]

    omop_concept_relationships['concept_id_1'] = omop_concept_relationships['concept_id_1'].astype(str)
    omop_concept_relationships['concept_id_2'] = omop_concept_relationships['concept_id_2'].astype(str)

    concept_maps = omop_concepts_filtered.merge( omop_concept_relationships, how='left', left_on='concept_id', right_on='concept_id_1' )
    concept_maps_named = concept_maps.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id' )

    target_codes_icd10 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD10PCS' ]
    target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
    print( target_codes_icd10['relationship_id'].value_counts() )

    target_codes_icd9 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD9Proc' ]
    target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
    print( target_codes_icd9['relationship_id'].value_counts() )

    target_codes_opcs4 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='OPCS4' ]
    target_codes_opcs4['opcs'] = target_codes_opcs4['concept_code_y'].str.replace('.','')

    print( target_codes_opcs4['relationship_id'].value_counts() )

    target_codes_icd9[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_icd9.csv')
    target_codes_icd10[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_icd10.csv')
    target_codes_opcs4[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv('codelists/output/omop/'+name+'_opcs.csv')

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:31: SettingWithCopyWarning: 
A value is trying t

relationship_id
Subsumes    2050
Name: count, dtype: int64
relationship_id
Mapped from    39
Subsumes       23
Name: count, dtype: int64
relationship_id
Mapped from    117
Subsumes        23
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:31: SettingWithCopyWarning: 
A value is trying t

relationship_id
Subsumes    1045
Name: count, dtype: int64
relationship_id
Mapped from    28
Subsumes       13
Name: count, dtype: int64
relationship_id
Mapped from    48
Subsumes       11
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:31: SettingWithCopyWarning: 
A value is trying t

relationship_id
Subsumes    730
Name: count, dtype: int64
relationship_id
Mapped from    25
Subsumes       13
Name: count, dtype: int64
relationship_id
Mapped from    49
Subsumes       10
Name: count, dtype: int64
relationship_id
Subsumes    268
Name: count, dtype: int64
relationship_id
Mapped from    18
Subsumes        9
Name: count, dtype: int64
relationship_id
Mapped from    26
Subsumes        5
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['opcs'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['opcs'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_1120/1421141509.py:31: SettingWithCopyWarning: 
A value is trying t

In [71]:
omop_concept_relationships[omop_concept_relationships['concept_id_1']=='4096461']

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
624715,4096461,46274039,Value mapped from,20220801,20991231,NaN
11186518,4096461,40642537,Has status,20220128,20991231,NaN
11186519,4096461,40642539,Has Module,20220128,20991231,NaN
14623737,4096461,2002767,Mapped from,20180625,20991231,NaN
14623738,4096461,2002768,Subsumes,20180625,20991231,NaN
14623739,4096461,2002769,Subsumes,20180625,20991231,NaN
14623740,4096461,2002770,Subsumes,20180625,20991231,NaN
14623741,4096461,4018020,Concept was_a from,20140401,20991231,NaN
14623742,4096461,4018020,Mapped from,19700101,20991231,NaN
14623743,4096461,4018161,Concept was_a from,20140401,20991231,NaN


In [72]:
method_relationships = omop_concept_relationships[omop_concept_relationships['relationship_id']=='Has method']
method_relationships

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
14106,4237501,4045073,Has method,20220128,20991231,NaN
20970,619843,40293773,Has method,20220128,20991231,NaN
97320,36674184,4044176,Has method,20190131,20991231,NaN
97337,36674186,4081836,Has method,20190131,20991231,NaN
97339,36674186,4106470,Has method,20190131,20991231,NaN
...,...,...,...,...,...,...
18892362,619793,4044525,Has method,20220128,20991231,NaN
18892369,619794,4044525,Has method,20220128,20991231,NaN
18892385,619795,40293775,Has method,20220128,20991231,NaN
18892400,618616,40293773,Has method,20220128,20991231,NaN


In [73]:
snomed2opcs = target_codes_opcs4[['concept_id_x','concept_name_x','concept_name_y','concept_id_y','concept_code_y']]

snomed2opcs

m1 = snomed2opcs.merge(method_relationships, how='left', left_on='concept_id_x', right_on='concept_id_1')
m1
m2 = m1.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id')
m2

methods = m2[['concept_name_x','concept_name_y','concept_code_y','concept_name']]

In [74]:
opcs2cat = pd.DataFrame(data={
    'opcs_name': methods['concept_name_y'],
    'opcs_code': methods['concept_code_y'],
    'category': methods['concept_name']
}).drop_duplicates()
opcs2cat

,opcs_name,opcs_code,category
0,Other specified excision of lung,E54.8,Excision - action
1,Unspecified excision of lung,E54.9,Excision - action
2,Excision of lung,E54,Excision - action
3,Bilobectomy of lung,E54.2,Excision - action
4,Lobectomy of lung,E54.3,Excision - action
5,Excision of segment of lung,E54.4,Excision - action
6,Other specified open extirpation of lesion of ...,E55.8,Extirpation - action
7,Unspecified open extirpation of lesion of lung,E55.9,Extirpation - action
8,Open extirpation of lesion of lung,E55,Extirpation - action
9,Open decortication of lesion of lung,E55.1,Decortication - action


In [75]:
opcs2cat['opcs_code'].value_counts()

opcs_code
E57.2    2
E54.8    1
T07.1    1
E53      1
E53.9    1
E53.8    1
U15.2    1
E54.1    1
E57.4    1
E55.4    1
E59.3    1
X45.3    1
E54.6    1
E59.5    1
E54.5    1
E53.2    1
E59.2    1
E54.9    1
E59.1    1
E57.3    1
E55.3    1
E55.2    1
E55.1    1
E55      1
E55.9    1
E55.8    1
E54.4    1
E54.3    1
E54.2    1
E54      1
E57.1    1
Name: count, dtype: int64